In [6]:
import sys
import asyncio
import time
import os

import numpy as np

from lsst.ts import salobj

from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.auxtel.latiss import LATISS

In [7]:
# for autocompleted to work
%config IPCompleter.use_jedi = False

In [8]:
import logging
stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [9]:
print(os.environ["OSPL_URI"])
print(os.environ["LSST_DDS_PARTITION_PREFIX"])

file:///home/patrickingraham/repos/ts_ddsconfig/config/ospl-sp.xml
summit


In [ ]:
#get classes and start them
domain = salobj.Domain()
await asyncio.sleep(10) # This can be removed in the future...
atcs = ATCS(domain)
latiss = LATISS(domain)
await asyncio.gather(atcs.start_task, latiss.start_task)

In [ ]:
await atcs.rem.atdome.cmd_moveShutterDropoutDoor.set_start(open=True)

In [10]:
tmp = atcs.rem.atptg.evt_summaryState.get()
print(salobj.State(tmp.summaryState))

State.STANDBY


In [11]:
tmp = await atcs.rem.atdome.evt_heartbeat.next(flush=True)
print(tmp)

private_revCode: 9816e66e, private_sndStamp: 1612286817.4665236, private_rcvStamp: 1612286817.4672172, private_seqNum: 2135938, private_identity: ATDome, private_origin: 10006, private_host: 0, heartbeat: False, priority: 0


In [ ]:
tmp =latiss.rem.atspectrograph.evt_settingsApplied.get()
print(tmp)

In [ ]:
await latiss.enable()

In [ ]:
tmp = latiss.rem.atarchiver.evt_heartbeat.get()
print(tmp)

In [ ]:
tmp = latiss.rem.atarchiver.evt_summaryState.get()
print(salobj.State(tmp.summaryState))

In [ ]:
# enable components
await atcs.enable()
#await latiss.standby()

In [20]:
# ATDOME only
tmp = await salobj.set_summary_state(atcs.rem.atdome, salobj.State.STANDBY, settingsToApply='test')
#tmp = await salobj.set_summary_state(atcs.rem.atdometrajectory, salobj.State.STANDBY)
#tmp=await atcs.rem.atpneumatics.cmd_standby.start()
#print(tmp)

In [ ]:
# ATMCS
tmp = await salobj.set_summary_state(atcs.rem.atmcs, salobj.State.STANDBY)
#tmp=The job description is a bit of a catch-all, if someone could provide a more detailed list of priorities and responsibilities I think that would help all parties.await atcs.rem.atmcs.cmd_start.start(timeout=5)

In [ ]:
# ATPneumatics
tmp = await salobj.set_summary_state(atcs.rem.atpneumatics, salobj.State.STANDBY)
#tmp=await atcs.rem.atpneumatics.cmd_start.start()

In [ ]:
# ATPtg
tmp = await salobj.set_summary_state(atcs.rem.atptg, salobj.State.STANDBY)
#tmp=await atcs.rem.atptg.cmd_enable.start()
# Need to manually say which rotator we're using at the moment (N2-->focus=3, N1-->focus=2)
#await atcs.rem.atptg.cmd_focusName.set_start(focus=3)

In [ ]:
# ATAOS
tmp = await salobj.set_summary_state(atcs.rem.ataos, salobj.State.STANDBY, settingsToApply='hex_m1_hex_202003_v3.yaml')
#tmp=await atcs.rem.ataos.cmd_enable.start()

In [13]:
await atcs.rem.atdome.cmd_homeAzimuth.start()

In [15]:
# This doesn't move anything! 
tmp=await atcs.rem.atdome.cmd_moveAzimuth.set_start(5)
print(tmp)

private_revCode: ec56b694, private_sndStamp: 1612287411.784614, private_rcvStamp: 1612287411.787763, private_seqNum: 919827472, private_identity: ATDome, private_origin: 10006, private_host: 0, ack: 303, error: 0, result: Done, host: 0, identity: patrickingraham@nb-patrickingraham-w-2021-02, origin: 16963, cmdtype: 8, timeout: 0.0


In [19]:
# This does
tmp=await atcs.rem.atdome.cmd_moveAzimuth.set_start(azimuth=0)
print(tmp)

private_revCode: ec56b694, private_sndStamp: 1612287735.4213793, private_rcvStamp: 1612287735.421914, private_seqNum: 919827476, private_identity: ATDome, private_origin: 10006, private_host: 0, ack: 303, error: 0, result: Done, host: 0, identity: patrickingraham@nb-patrickingraham-w-2021-02, origin: 16963, cmdtype: 8, timeout: 0.0


In [ ]:
# ATHexapod
tmp = await salobj.set_summary_state(atcs.rem.athexapod, salobj.State.STANDBY, settingsToApply='summit')
#tmp=await atcs.rem.athexapod.cmd_start.set_start(settingsToApply='summit')
#tmp=await atcs.rem.athexapod.cmd_enable.start()
print(tmp)

In [ ]:
# ATSPECTROGRAPH
tmp = await salobj.set_summary_state(latiss.rem.atspectrograph, salobj.State.ENABLED, settingsToApply='LATISS_AuxTel_20210108_v3.yaml')
#tmp=await latiss.rem.atspectrograph.cmd_start.set_start(settingsToApply='LATISS_AuxTel_20210108.yaml')
#tmp=await latiss.rem.atspectrograph.cmd_enable.start()
print(tmp)

In [ ]:
# take event checking out the slew commands to test telescope only
atcs.check.atdome = False
atcs.check.atdometrajectory = False

In [ ]:
offset = 1.12 #2./3600
start_az=205+offset
start_el=71+offset
start_rot_pa=1.5+offset
await atcs.point_azel(start_az, start_el, rot_tel=start_rot_pa, wait_dome=False)

In [ ]:
# Perform Rotator verification (LTS-337-003)True

In [ ]:
await atcs.rem.atmcs.cmd_abort.start()

In [ ]:
await atcs.stop_tracking()

In [ ]:
start_az=-58
start_el=35
start_rot_pa=13.1
await atcs.point_azel(start_az, start_el, rot_tel=start_rot_pa, wait_dome=False)

In [ ]:
start_az=165.5
start_el=67.9
start_rot_pa=170

start_time =time.time()
await atcs.point_azel(start_az, start_el, rot_pa=start_rot_pa, wait_dome=True)
end_time = time.time()
print(f'Time to perform {170*2} degree rotation of N2 is {end_time-start_time} seconds')

In [ ]:
# send telescope to alt/az/
#await salobj.set_summary_state(latiss.atcamera, salobj.State.ENABLED)

In [ ]:
# point telescope to desired starting position
# this make take extra time as more checks are performed
start_az=200
start_el=71
start_rot_pa=0
await atcs.point_azel(start_az, start_el, rot_tel=start_rot_pa, wait_dome=True)

In [ ]:
#declare offset size
d_az= 3.5 # degrees
d_el = 3.5 # degrees
d_rot= 3.5 # degrees

d_az= 10 # degrees
d_el = 10 # degrees
d_rot= 10 # degrees

d_az= 90 # degrees
d_el = 60 # degrees
d_rot= 90 # degrees

In [ ]:
await atcs.stop_tracking()

In [ ]:
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print(f"Current Time = {current_time}, or {time.time()}")

In [ ]:
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print(f"Current Time = {current_time}, or {time.time()}")

# Perform tracking test by taking continous 20s exposures
#await attcs.slew_object('HD 167060')
#await atcs.slew_object('HIP 100787')
await atcs.slew_object('51 Cap')

now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print(f"Current Time = {current_time}, or {time.time()}")

In [ ]:
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print(f"Current Time = {current_time}, or {time.time()}")
await atcs.stop_tracking()

In [ ]:
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print(f"Current Time = {current_time}, or {time.time()}")
await atcs.stop_tracking()

In [ ]:
await latiss.take_object(exptime=15, grating='empty_1', filter='RG610')

In [ ]:
await latiss.take_object(exptime=15, grating='ronchi90lpmm', filter='empty_1')

In [ ]:
while True:
    await latiss.take_object(exptime=15, grating='empty_1', filter='RG610')
    await latiss.take_object(exptime=15, grating='ronchi90lpmm', filter='empty_1')

In [ ]:
# stopped tracking elsewhere, then reslewed
await atcs.slew_object('HR 7474')
#await atcs.slew_object('HIP 23172')

In [ ]:
#await atcs.rem.ataos.cmd_setLogLevel.set_start(level=10)

In [ ]:
# OFFSET  AZ/EL
await attcs.offset_azel(az=-5*60*60, el=0)
await attcs.offset_azel(az=-2*60*60, el=0)
await attcs.offset_azel(az=0, el=0)

In [ ]:
while True:
    await latiss.take_object(exptime=15, grating='empty_1', filter='RG610')
    await latiss.take_object(exptime=15, grating='ronchi90lpmm', filter='empty_1')
    

In [ ]:
await latiss.take_bias(nbias=1)

In [ ]:
#await attcs.startup()

In [ ]:
await latiss.rem.atarchiver.cmd_start.start()

In [ ]:
# 30 Az, 71 El

In [ ]:
await atcs.slew_object('2 Cap')

In [ ]:
await atcs.slew_object('HD 185928')

In [ ]:
await atcs.slew_object('HD176614') # 81 elevation

In [ ]:
await atcs.slew_object('HD 181010',rot_type=4) # 84 elevation

In [ ]:
await atcs.slew_object('HR 6960',rot_type=4) # ~72 elevation

In [ ]:
await atcs.slew_object('HR 6334',rot_type=4) # 56 elevation

In [ ]:
await atcs.slew_object('5 Lup',rot_type=4) # ~40 elevation - 9:18 am - hit mount fault - velocity error in azimuth - possibly a slip?

In [ ]:
await atcs.slew_object('HD 185975',rot_type=4) # ~40 elevation - 9:18 am - hit mount fault - velocity error in azimuth - possibly a slip?

In [ ]:
await atcs.slew_object('HIP 101090') # az 30, el 80 -- going to try offset testing

In [ ]:
global currvalue

In [ ]:
currvalue = 0
offset=10
await atcs.offset_azel(offset,0, relative=True)
currvalue = currvalue + offset

In [ ]:
offset=-10
await atcs.offset_azel(offset,0, relative=True)
currvalue = currvalue + offset

In [ ]:
await atcs.offset_azel(0, offset, relative=True)

In [ ]:
await atcs.offset_azel(0, -offset, relative=True)

In [ ]:
await atcs.offset_xy(x=-offset, y=0, relative=True)

In [ ]:
await atcs.stop_tracking()

In [ ]:
#Hexapod offset : [ 0.03945005 -0.15717775 0.06446582]
await atcs.rem.ataos.cmd_offset.set_start(z=0.06446582)

In [ ]:
# -0.060342  - but need to remove this
await atcs.rem.ataos.cmd_offset.set_start(z=0.060342)

In [ ]:
# turn on corrections
tmp = await atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True, hexapod=True, atspectrograph=True)

In [ ]:
# turn off corrections
tmp = await atcs.rem.ataos.cmd_disableCorrection.set_start(m1=True, hexapod=True, atspectrograph=True)

In [ ]:
# is ATAOS correcting?
tmp = atcs.rem.ataos.evt_correctionEnabled.get()
print(tmp)

In [ ]:
# what is the M1 mirror pressure
pres = atcs.rem.ataos.evt_m1CorrectionStarted.get()
print(pres)

In [ ]:
# Verify that ATAOS handles focus/pointing offsets from grating change
await latiss.rem.atspectrograph.cmd_changeFilter.set_start(filter=1)
await latiss.rem.atspectrograph.cmd_changeDisperser.set_start(disperser=1)

In [ ]:
await latiss.setup_atspec

In [ ]:
# change ataos wavelength
tmp = await atcs.rem.ataos.cmd_setWavelength.set_start(wavelength=700)
print(tmp)

In [ ]:
# get offsets from ATAOS - after removing artificial offset
focus_offsets = atcs.rem.ataos.evt_focusOffsetSummary.get()
print(focus_offsets)

In [ ]:
# get offsets from ATAOS - after turning on atspectrograph correction
focus_offsets = atcs.rem.ataos.evt_focusOffsetSummary.get()
print(focus_offsets)

In [ ]:
# get offsets from ATAOS - after reset
focus_offsets = await atcs.rem.ataos.evt_focusOffsetSummary.aget()
print(focus_offsets)

In [ ]:
# now apply user offset
await atcs.rem.ataos.cmd_offset.set_start(z=0.032214134931564)

In [ ]:
# get offsets from ATAOS 
focus_offsets = await atcs.rem.ataos.evt_focusOffsetSummary.aget()
print(focus_offsets)

In [ ]:
# get pointing offsets from ATAOS
pointing_offsets = await atcs.rem.ataos.evt_pointingOffsetSummary.aget()
print(pointing_offsets)

In [ ]:
# now apply user offset
await atcs.rem.ataos.cmd_offset.set_start(z=1.34)

In [ ]:
# get pointing offsets from ATAOS
pointing_offsets = atcs.rem.ataos.evt_pointingOffsetSummary.get()
print(pointing_offsets)

In [ ]:
# reset offsets
tmp = await atcs.rem.ataos.cmd_resetOffset.set_start(axis='z')

In [ ]:
# take LATISS images
await latiss.take_bias(nbias=30)

In [ ]:
# get offsets from ATAOS
focus_offsets = await atcs.rem.ataos.evt_focusOffsetSummary.aget()
print(focus_offsets)